# <center>Curricular Analtyics Applied to Transfer Articulation: A Case Study</center>

<center>
    <b>Hayden W. Free$^\ddagger$, Gregory L. Heileman,$^\star$ and Chaouki T. Abdallah$^\bullet$</b> <br><br>
    $^\star$Department of Electrical & Computer Engineering <br>
    University of Arizona <br>
    heileman@arizona.edu <br><br>
    $^\ddagger$Department of Computer Science <br>
    University of Kentucky <br>
    hayden.free@uky.edu<br><br>
    $^\bullet$Department of Electrical & Computer Engineering <br>
    Georgia Institute of Technology <br>
    ctabdallah@gatech.edu
</center>

The Curricular Analytics Toolbox includes capablities that allow you to create customized degree plans according to various user-specifed optimization criteria. In order to use these capabilities, you first must have permission to access the CurricularOptimziation.jl package through a professional-level account. To obtain a professional-level account, please visit: http://CurricularAnalytics.org/pro. 

After obtaining your professional account, in order to use the CurricularOptimization tools, first enter package mode from the Julia REPL by pressing `]`, then enter: 

`pkg> add https://github.com/CurricularAnalytics/CurricularOptimization.jl.git`

The curricular optimization features make use of the [JuMP](https://github.com/JuliaOpt/JuMP.jl) domain-specific language for specifying optimization problems in Julia, and call the [Gurobi](https://www.gurobi.com) solver in order to solve the optimzaton problems. In order to use these features you must also install the solver, called [Gurobi Optimizer](https://www.gurobi.com/downloads/gurobi-optimizer-eula). Gurobi is a commercial product, and requires a license key; however, [academic licenses](https://www.gurobi.com/downloads/end-user-license-agreement-academic) are available at no cost.

After installing the Gurobi Solver on your computer, execute the following commands to use the optimization capabilities within the Curricular Analytics Toolbox: 

In [1]:
using CurricularAnalytics, CurricularVisualization, CurricularOptimization, JuMP

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-11576338707305104579\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-13950933209811820654\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-11576338707305104579\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

# Introduction
What makes this problem hard? 
- Credit recognition - how will the courses be counted once tranferred?
- Double counting courses that satisfy major & gen ed requirements

# Example - Two Students Attempt to Study Basket Weaving
In the following example we demonstrate two students that take very similiar courses, but one student transfers from a community college. This example highlights the large difference in degree progress that can occur depending on how courses transfer.

For the sake of conserving space most of the input to perform these steps is contained in the `basket-weaving-example.jl` file, which is included with this notebook. Snippets of the various actions are highlighted below.

```julia
# Create stores for institutions courses and catalog.
college = "KCTCS"
univ = "Northern Keuncky University"
KCTCS_catalog = CourseCatalog("2019-20 Academic Year", college)
NKU_catalog = CourseCatalog("2019-20 Academic Year", univ)
KCTCS_courses = Array{Course,1}()
NKU_courses = Array{Course,1}()

# Create the KCTCS Courses 
push!(KCTCS_courses, Course("Writing I", 3, prefix="ENG", num="101", institution=college))
...
# Add KCTCS courses to catalog
add_course!(KCTCS_catalog, KCTCS_courses)

# NKU Courses
push!(NKU_courses, Course("Great Archaeological Sites", 3, prefix="ANT", num="114", institution=univ))
...
# Add NKU courses to catalog
add_course!(NKU_catalog, NKU_courses)

# NKU General Education Requirement
# First Communication subrequirement
opt1 = CourseSet("Written Option 1", 3, [course(NKU_catalog, "ENG", "104", "Honors College Writing") => grade("D")])
...
# Construct full communication requirement set
comm = RequirementSet("Communication", 9, [written, oral], description="Gen. Ed. sub-requirement")

# Create CS Major Requirements
cs_dr = RequirementSet("Degree Requirements", 120, convert(Array{AbstractRequirement,1}, [gen_ed, cs_mr]), description="Degree Requirements for the BS Computer Science Degree")

# Create the transfer articulation map between KCTCS and NKU
NKU_xfer = TransferArticulation("NKU Transfer Articulation", univ, NKU_catalog)
add_transfer_catalog(NKU_xfer, KCTCS_catalog)
add_transfer_course(NKU_xfer, [course_id("ENG", "101", "College Writing", univ)], KCTCS_catalog.id, course_id("ENG", "101", "Writing I", college))
...

# Create a transcript involving KCTCS coursework
KCTCS_transcript = Array{CourseRecord,1}()
push!(KCTCS_transcript, CourseRecord(course(KCTCS_catalog, "CIT", "105", "Introduction to Computing"), grade("A")))
...

# Finally use the NKU_xfer object to find equivalent courses for all courses in KCTCS_transcript
```

In [2]:
include("./basket-weaving-example.jl")
display("text/html", "<style>.container { width:100% !important; }</style>")

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 17 rows, 32 columns and 154 nonzeros
Model fingerprint: 0xd8c700fc
Model has 2 general constraints
Variable types: 0 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 17 rows, 32 columns and 154 nonzeros
Model has 2 general constraints
Variable types: 0 continuous, 32 inte

LoadError: LoadError: UndefVarError: institution not defined
in expression starting at /Users/heileman/Library/Mobile Documents/com~apple~CloudDocs/work/research/Curricular Analytics/CA-Notebooks/Requirement Analytics/Basket Weaving/basket-weaving-example.jl:131